In [ ]:
import pandas as pd
import numpy as np
import nltk
import string

from nltk.corpus import stopwords
from ast import literal_eval

data = pd.read_csv("text_emotion2.csv")

from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def word_clean(key):
    key_words = data[data.sentiment == key]["content"].str.cat(sep="\n")
    tokens = nltk.word_tokenize(key_words)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in words]
    stemmed = set([word for word in stemmed if len(word) > 2])
    return (words)

anxiety_words = word_clean("anxiety")
enth_words = word_clean("enthusiasm")
love_words = word_clean("love")
happy_words = word_clean("happiness")
anxiety_trig = ["anxious", "stressed", "sweating", "nervous", "nausea", "dizzy", "shakiness", "blushed", "sleep", "escape"]

anx_words = [word for word in anxiety_words if word not in enth_words]
anx_words.extend([porter.stem(word) for word in anxiety_trig])

urgent = ["suicide", "kill", "end", "die", "suicidal"]
urgent = set([porter.stem(word) for word in urgent])

def simple_tokenize(s):
    return re.findall(r"[a-z]+(?:'[a-z]+)?",s.lower())

def response(user_response):
    words = simple_tokenize(user_response)
    for word in words:
        if porter.stem(word) in urgent:
            return "URGENT"
    
    return "NON-URGENT"

def sentiment (user_response):
    tokens = simple_tokenize(user_response)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    words = set(words)
    
    
    total_len = len(words)
    count = 0
    for word in words:
        if porter.stem(word) in anx_words:
            count += 1
    
    if count/total_len >= 0.3:
        return "ANXIETY"
    else:
        return "NON-ANXIETY"

kindwords = pd.read_csv("kindwords.csv")

import time
import random

CALMING_ACTIVITY = []
REASON_TO_LIVE = []
FRIEND_PHONE_NUMBER = ""
BOT_NAME = ""

KIND_WORDS = kindwords["Supportive phrase"].tolist()

print ("Hi, I am your artificial companion, how would you like to call me?")
BOT_NAME = input()
print ("Please list three things that will calm you down")
CALMING_ACTIVITY.append(input("thing1"))
CALMING_ACTIVITY.append(input("thing2"))
CALMING_ACTIVITY.append(input("thing3"))

print ("Please list three reasons to make you feel your life is meaningful")
CALMING_ACTIVITY.append(input("thing1"))
CALMING_ACTIVITY.append(input("thing2"))
CALMING_ACTIVITY.append(input("thing3"))

print ("Please give me the phone number of your trusted friend")
FRIEND_PHONE_NUMBER = ""


while True:
    print ("Hi, this is {0}. How do you feel today".format(BOT_NAME))
    user_response = input()
    
    if user_response != "Bye":
        
        if response(user_response) == "URGENT":
            print ("Would you like me to call your friend?")
            user_response = input()
            if user_response == "yes":
                print ("Calling {0}".format(FRIEND_PHONE_NUMBER))
            else:
                print ("I'd recommend you to talk to some one because I'm worried about you...")
                print ("...")
                print ("Would you like to do {0}".format(CALMING_ACTIVITY[0]))

        else:
            if sentiment(user_response) == "ANXIETY":
                bot_response = random.choice(KIND_WORDS) + "\nplease do some active breathing activity with me..."
                print (bot_response)
                print ("Inhale and count 1...")
                time.sleep(5)
                print ("Hold it 2...")
                time.sleep(5)
                print ("Exhale 3...")
                time.sleep(5)
                print("Would you like to do more excercise with me?y/n")
                user_response = input()
    else:
        print("Take care and talk to me again")
        break
    
    
    
    

